In [30]:
import pandas as pd

df = pd.read_csv(r'C:\Users\clint\Desktop\RA Task\5.csv')
df['year'] = df['year'].astype('Int64')
df

,state,county,fips,year,project_start,project_id,route,mileage,lanes,project_duration_days,eng_estimate_mils,win_bid_mils,cost_mils,num_bidders,bidders_list,all_routes,cost_overrun_pct
0,Ohio,Paulding,39125,2018,2018-05-24 00:00:00,105522,111,12.982,2.0,99.0,0.943,0.957859,1.047510,2.0,"Shelly Company, Gerken Paving",NaN,9.359511
1,Ohio,Wyandot,39175,2018,2018-11-15 00:00:00,88832,23,NaN,4.0,290.0,NaN,3.236775,3.304783,NaN,NaN,"['23', '23', '23']",2.101101
2,Ohio,Butler,39017,2018,2018-01-18 00:00:00,94263,73,NaN,2.0,195.0,NaN,0.258900,0.232678,NaN,NaN,['73'],-10.128362
3,Ohio,Franklin,39049,2018,2018-01-18 00:00:00,76467,270 / 315,NaN,4.0,255.0,NaN,6.101481,6.991613,NaN,NaN,"['270', '270', '270', '270', '270', '270', '27...",14.588788
4,Ohio,Hocking,39073,2018,2018-01-18 00:00:00,101555,33,NaN,2.0,255.0,NaN,0.553756,0.531749,NaN,NaN,['33'],-3.974212
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
198,Ohio,Harrison,39067,2018,2018-12-20 00:00:00,91844,250 / 9,NaN,2.0,284.0,NaN,2.284000,2.212447,NaN,NaN,"['9', '250']",-3.132812
199,Ohio,Highland,39071,2018,2018-12-20 00:00:00,84622,138 / 753,NaN,2.0,223.0,NaN,1.494436,1.658765,NaN,NaN,"['138', '138', '138', '753']",10.996011
200,Ohio,Madison,39097,2018,2018-12-20 00:00:00,105547,42,NaN,2.0,284.0,NaN,3.611668,3.535001,NaN,NaN,"['42', '42', '42']",-2.122762
201,Ohio,Clinton,39027,2018,2018-12-20 00:00:00,87300,251 / 68 / 350,NaN,2.0,589.0,NaN,5.441520,5.331566,NaN,NaN,"['68', '68', '68', '68', '68', '68', '251', '3...",-2.020640


In [31]:
# Import required libraries
import os
import time
import requests
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager

In [32]:
# Define function to download both BidTab and Proposal for a given PID
def download_bid_documents(pid_number, wait_time=5):
    """
    Download both BidTab and Proposal documents for a given PID number.
    
    Parameters:
    -----------
    pid_number : str
        The Project ID number to search for
    wait_time : int, optional
        Time to wait between actions (default: 5)
        
    Returns:
    --------
    tuple
        (bidtab_success, proposal_success) indicating whether each download was successful
    """
    # Dictionary to track download success
    results = {
        'bidtab': False,
        'proposal': False,
        'bidtab_file': None,
        'proposal_file': None
    }
    
    # Create downloads directory if it doesn't exist
    download_dir = r"C:\Users\clint\Desktop\RA Task\Downloads"
    if not os.path.exists(download_dir):
        os.makedirs(download_dir)
        print(f"Created directory: {download_dir}")
        
    # Get list of existing PDF files before download
    existing_pdfs = set(glob.glob(os.path.join(download_dir, "*.pdf")))
    
    try:
        # Set up Chrome options
        chrome_options = Options()
        chrome_options.add_experimental_option("prefs", {
            "download.default_directory": download_dir,
            "download.prompt_for_download": False,
            "plugins.always_open_pdf_externally": True  # Force PDF files to download instead of open in browser
        })
        # Add argument to start browser maximized
        chrome_options.add_argument("--start-maximized")
        
        # Initialize the WebDriver
        driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
        wait = WebDriverWait(driver, 20)
        
        # Download BidTab
        try:
            print(f"\nDownloading BidTab for PID {pid_number}...")
            
            # Navigate to the Ohio DOT contracts website
            driver.get("https://contracts.dot.state.oh.us/document/documentSearchCriteria.do?from=topNav&newSearch=true")
            
            # Select the "BidTabs" option
            bid_tabs_dropdown = wait.until(EC.presence_of_element_located((By.XPATH, "//select[option[@value='1607']]")))
            bid_tabs_dropdown.click()
            bid_tabs_option = wait.until(EC.presence_of_element_located((By.XPATH, "//option[@value='1607']")))
            bid_tabs_option.click()
            
            # Enter the PID number
            pid_input = wait.until(EC.presence_of_element_located((By.NAME, "DP.PID_NUM.TEXT")))
            pid_input.send_keys(pid_number)
            
            # Click the search button
            try:
                # Try multiple methods to find and click search button
                try:
                    search_button = driver.find_element(By.XPATH, "//input[@type='submit' and @class='blue-button']")  
                except Exception:
                    try:
                        search_button = driver.find_element(By.CLASS_NAME, "blue-button")
                    except Exception:
                        search_button = driver.find_element(By.XPATH, "//input[@type='submit']")
                        
                driver.execute_script("arguments[0].scrollIntoView(true);", search_button)
                search_button.click()
                
                # Wait for results
                time.sleep(wait_time)
                
                # Check if results found
                if "No Records Found" in driver.page_source:
                    print(f"No BidTab found for PID {pid_number}")
                else:
                    # Click on the View icon if available
                    try:
                        view_icon = wait.until(EC.presence_of_element_located((By.XPATH, "//img[@src='/images/open.gif']")))
                        view_icon.click()
                        print(f"BidTab download started for PID {pid_number}")
                        time.sleep(wait_time)  # Wait for download to start
                        results['bidtab'] = True
                    except Exception as e:
                        print(f"Error clicking view icon for BidTab: {e}")
            except Exception as e:
                print(f"Error searching for BidTab: {e}")
        except Exception as e:
            print(f"Error in BidTab download process: {e}")
            
        # Download Proposal
        try:
            print(f"\nDownloading Proposal for PID {pid_number}...")
            
            # Navigate back to search page
            driver.get("https://contracts.dot.state.oh.us/document/documentSearchCriteria.do?from=topNav&newSearch=true")
            
            # Select the "Proposals" option
            proposals_dropdown = wait.until(EC.presence_of_element_located((By.XPATH, "//select[option[@value='1003']]")))
            proposals_dropdown.click()
            proposals_option = wait.until(EC.presence_of_element_located((By.XPATH, "//option[@value='1003']")))
            proposals_option.click()
            
            # Enter the PID number
            pid_input = wait.until(EC.presence_of_element_located((By.NAME, "DP.PID_NUM.TEXT")))
            pid_input.send_keys(pid_number)
            
            # Click the search button
            try:
                # Try multiple methods to find and click search button
                try:
                    search_button = driver.find_element(By.XPATH, "//input[@type='submit' and @class='blue-button']")  
                except Exception:
                    try:
                        search_button = driver.find_element(By.CLASS_NAME, "blue-button")
                    except Exception:
                        search_button = driver.find_element(By.XPATH, "//input[@type='submit']")
                        
                driver.execute_script("arguments[0].scrollIntoView(true);", search_button)
                search_button.click()
                
                # Wait for results
                time.sleep(wait_time)
                
                # Check if results found
                if "No Records Found" in driver.page_source:
                    print(f"No Proposal found for PID {pid_number}")
                else:
                    # Click on the View icon if available
                    try:
                        view_icon = wait.until(EC.presence_of_element_located((By.XPATH, "//img[@src='/images/open.gif']")))
                        view_icon.click()
                        print(f"Proposal download started for PID {pid_number}")
                        time.sleep(wait_time)  # Wait for download to start
                        results['proposal'] = True
                    except Exception as e:
                        print(f"Error clicking view icon for Proposal: {e}")
            except Exception as e:
                print(f"Error searching for Proposal: {e}")
        except Exception as e:
            print(f"Error in Proposal download process: {e}")
        
        # Close the browser
        driver.quit()
        print("Browser closed")
        
        # Wait a moment for downloads to complete
        time.sleep(wait_time)
        
        # Check for new PDF files
        new_pdfs = set(glob.glob(os.path.join(download_dir, "*.pdf"))) - existing_pdfs
        if new_pdfs:
            print(f"Downloaded {len(new_pdfs)} new file(s):")
            for pdf in new_pdfs:
                if pid_number in pdf.lower() and "bidtab" in pdf.lower():
                    results['bidtab_file'] = pdf
                    print(f"BidTab file: {os.path.basename(pdf)}")
                elif pid_number in pdf.lower():
                    results['proposal_file'] = pdf
                    print(f"Proposal file: {os.path.basename(pdf)}")
                else:
                    print(f"Downloaded file: {os.path.basename(pdf)}")
        else:
            print("No new files were downloaded")
            
    except Exception as e:
        print(f"An error occurred: {e}")
        
    return results

In [33]:
# Set up Chrome options
chrome_options = Options()
chrome_options.add_experimental_option("prefs", {
    "download.default_directory": r"C:\Users\clint\Desktop\RA Task\Downloads",
    "download.prompt_for_download": False,
    "plugins.always_open_pdf_externally": True  # Force PDF files to download instead of open in browser
})
# Add argument to start browser maximized (fullscreen)
chrome_options.add_argument("--start-maximized")

# Initialize the WebDriver
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)

# Additionally, maximize the window to ensure fullscreen
driver.maximize_window()

In [34]:
# Navigate to the Ohio DOT contracts website
driver.get("https://contracts.dot.state.oh.us/document/documentSearchCriteria.do?from=topNav&newSearch=true")

# Wait for the page to load completely
wait = WebDriverWait(driver, 20)

# First, select the "BidTabs" option from the dropdown
try:
    print("Step 1: Looking for the dropdown containing BidTabs option...")
    # Look for the dropdown containing BidTabs option - typically this would be in a select element
    bid_tabs_dropdown = wait.until(EC.presence_of_element_located((By.XPATH, "//select[option[@value='1607']]")))
    print("Found the dropdown, clicking...")
    
    bid_tabs_dropdown.click()
    print("Clicked on the dropdown")
    
    # Select the BidTabs option
    print("Step 2: Looking for the BidTabs option...")
    bid_tabs_option = wait.until(EC.presence_of_element_located((By.XPATH, "//option[@value='1607']")))
    print("Found the BidTabs option, clicking...")
    
    bid_tabs_option.click()
    print("Successfully selected BidTabs option")
    
    # Find the PID input field and enter the PID number 
    print("Step 3: Looking for the PID input field...")
    pid_input = wait.until(EC.presence_of_element_located((By.NAME, "DP.PID_NUM.TEXT")))
    print("Found the PID input field, entering PID number...")
    
    # Using consistent PID number (was showing 88832 in the print message but entering 76467)
    pid_number = "76467"
    pid_input.send_keys(pid_number)
    print(f"Successfully entered PID number {pid_number}")
    
    # Click the search button (usually at the bottom of the form)
    print("Step 4: Looking for the search button...")
    
    # Take a screenshot of the page for analysis
    driver.save_screenshot(r"C:\Users\clint\Desktop\RA Task\Downloads\page_before_search.png")
    print("Saved screenshot of page before search attempt")
    
    print("Attempting multiple methods to find and click the search button...")
    
    # Method 1: Try with the exact HTML you provided
    try:
        print("Method 1: Using exact attributes from HTML")
        # Use a more precise selector that matches the exact HTML structure you shared
        search_button = driver.find_element(By.XPATH, "//input[@type='submit' and @class='blue-button' and contains(@value, 'Search') and @onclick='return submitAction(this.form);']")
        print("Found button using exact HTML attributes")
    except Exception as e:
        print(f"Method 1 failed: {e}")
        # Method 2: Try each attribute separately
        try:
            print("Method 2: Using class attribute only")
            search_button = driver.find_element(By.CLASS_NAME, "blue-button")
            print("Found button by class name")
        except Exception as e:
            print(f"Method 2 failed: {e}")
            try:
                print("Method 3: Using onclick attribute")
                search_button = driver.find_element(By.XPATH, "//input[@onclick='return submitAction(this.form);']")
                print("Found button by onclick attribute")
            except Exception as e:
                print(f"Method 3 failed: {e}")
                try:
                    print("Method 4: Using any submit button")
                    search_button = driver.find_element(By.XPATH, "//input[@type='submit']")
                    print("Found a generic submit button")
                except Exception as e:
                    print(f"Method 4 failed: {e}")
                    print("All element-finding methods failed, will try position-based clicking")
                    # We'll set search_button to None to indicate we need to use position-based clicking
                    search_button = None
    
    # Take a screenshot before clicking
    driver.save_screenshot(r"C:\Users\clint\Desktop\RA Task\Downloads\before_search_click.png")
    print(f"Took screenshot before clicking - check 'before_search_click.png'")
    time.sleep(2)
    
    if search_button:
        # If we found the button element, try regular clicking methods
        print("Button element found. Attempting to click it...")
        # Scroll to make sure the button is visible
        driver.execute_script("arguments[0].scrollIntoView(true);", search_button)
        print("Scrolled to the button")
        time.sleep(2)
        
        try:
            search_button.click()
            print("Clicked the button using normal click")
            click_success = True
        except Exception as click_error:
            print(f"Normal click failed: {click_error}")
            try:
                # Try JavaScript click
                driver.execute_script("arguments[0].click();", search_button)
                print("Clicked button using JavaScript")
                click_success = True
            except Exception as js_error:
                print(f"JavaScript click failed: {js_error}")
                click_success = False
    else:
        click_success = False
    
    # If button element methods failed, try position-based clicking
    if not search_button or not click_success:
        print("Attempting position-based clicking...")
        
        # Method 5: Try using ActionChains to move to bottom of form and click
        try:
            print("Method 5: Using ActionChains to move near bottom of page and click")
            from selenium.webdriver.common.action_chains import ActionChains
            
            # Get the form element or body if no form
            try:
                form = driver.find_element(By.XPATH, "//form")
            except:
                form = driver.find_element(By.TAG_NAME, "body")
            
            # Calculate a position where the search button might be (typically bottom of form)
            form_rect = form.rect
            action = ActionChains(driver)
            
            # Move to the bottom of the form
            action.move_to_element_with_offset(
                form, 
                form_rect['width'] // 2,  # Middle horizontally
                form_rect['height'] - 50  # Near the bottom
            )
            
            # Take a screenshot showing the position
            driver.save_screenshot(r"C:\Users\clint\Desktop\RA Task\Downloads\position_before_click.png")
            print("About to click at calculated position")
            time.sleep(2)
            
            # Click at that position
            action.click().perform()
            print("Clicked at the calculated position")
            click_success = True
        except Exception as pos_click_error:
            print(f"Position-based click failed: {pos_click_error}")
            click_success = False
    
    # If all else fails, try direct form submission
    if not click_success:
        try:
            print("Method 6: Direct form submission")
            form = driver.find_element(By.TAG_NAME, "form")
            form.submit()
            print("Submitted the form directly")
        except Exception as form_error:
            print(f"Form submission failed: {form_error}")
            raise Exception("All methods to submit the search failed")
    
    print("Search action attempted, waiting for results to load...")
    time.sleep(8)  # Extended pause to ensure page loads
    
    # Take a screenshot after search to verify the results page
    driver.save_screenshot(r"C:\Users\clint\Desktop\RA Task\Downloads\after_search_click.png")
    print(f"Took screenshot after search - check 'after_search_click.png'")
    
    # Check if we're on the results page
    try:
        # Try to find elements that would only be on the results page
        results_check = driver.find_elements(By.XPATH, "//table//tr")
        if len(results_check) > 1:
            print(f"Success! Found {len(results_check)} rows in results table")
        else:
            print("Results page may have loaded but no results found")
        
        # Check page URL or title to confirm we navigated
        current_url = driver.current_url
        print(f"Current URL: {current_url}")
        if "documentSearchResults" in current_url:
            print("URL confirms we're on the results page")
        
        page_source = driver.page_source
        if "No Records Found" in page_source:
            print("Warning: 'No Records Found' text detected on page")
        if pid_number in page_source:
            print(f"Success! PID number {pid_number} found in results")
    except Exception as result_check_error:
        print(f"Error checking results page: {result_check_error}")
    
    # Click on the "View" icon to download the PDF
    print("Step 5: Looking for the 'View' icon...")
    view_icon = wait.until(EC.presence_of_element_located((By.XPATH, "//img[@src='/images/open.gif']")))  
    print("Found the 'View' icon, clicking to download PDF...")
    
    view_icon.click()
    print("Clicked the view icon, PDF download started...")
    
    # Wait for the download to begin
    time.sleep(5)
    
except Exception as e:
    print(f"An error occurred: {e}")

Step 1: Looking for the dropdown containing BidTabs option...
Found the dropdown, clicking...
Clicked on the dropdown
Step 2: Looking for the BidTabs option...
Found the BidTabs option, clicking...
Successfully selected BidTabs option
Step 3: Looking for the PID input field...
Found the PID input field, entering PID number...
Successfully entered PID number 76467
Step 4: Looking for the search button...
Saved screenshot of page before search attempt
Attempting multiple methods to find and click the search button...
Method 1: Using exact attributes from HTML
Found button using exact HTML attributes
Successfully entered PID number 76467
Step 4: Looking for the search button...
Saved screenshot of page before search attempt
Attempting multiple methods to find and click the search button...
Method 1: Using exact attributes from HTML
Found button using exact HTML attributes
Took screenshot before clicking - check 'before_search_click.png'
Took screenshot before clicking - check 'before_sear

In [35]:
# Verify the PDF downloads
import glob
import os
import time

# Set the download directory
download_dir = r"C:\Users\clint\Desktop\RA Task\Downloads"

# Create the Downloads directory if it doesn't exist
if not os.path.exists(download_dir):
    os.makedirs(download_dir)
    print(f"Created directory: {download_dir}")

# Wait a bit for the downloads to complete
print("Waiting for downloads to complete...")
time.sleep(8)

# Get list of PDF files in the download directory
pdf_files = glob.glob(os.path.join(download_dir, "*.pdf"))

# Sort PDF files by modification time (newest first)
pdf_files.sort(key=os.path.getmtime, reverse=True)

# Check the two most recent PDFs (should be BidTab and Proposal)
if len(pdf_files) >= 2:
    # Most recent (should be Proposal)
    proposal_pdf = pdf_files[0]
    print(f"Proposal PDF downloaded: {proposal_pdf}")
    print(f"File size: {os.path.getsize(proposal_pdf) / 1024:.2f} KB")
    print(f"Download time: {time.ctime(os.path.getmtime(proposal_pdf))}")
    
    # Second most recent (should be BidTab)
    bidtab_pdf = pdf_files[1]
    print(f"\nBidTab PDF downloaded: {bidtab_pdf}")
    print(f"File size: {os.path.getsize(bidtab_pdf) / 1024:.2f} KB")
    print(f"Download time: {time.ctime(os.path.getmtime(bidtab_pdf))}")
    
    print(f"\nTotal PDFs in download directory: {len(pdf_files)}")
elif len(pdf_files) == 1:
    most_recent_pdf = pdf_files[0]
    print(f"Only one PDF downloaded: {most_recent_pdf}")
    print(f"File size: {os.path.getsize(most_recent_pdf) / 1024:.2f} KB")
    print(f"Download time: {time.ctime(os.path.getmtime(most_recent_pdf))}")
else:
    print("No PDF files found in the download directory")

Waiting for downloads to complete...
Only one PDF downloaded: C:\Users\clint\Desktop\RA Task\Downloads\180012bidtab.pdf
File size: 292.88 KB
Download time: Sun Jun  1 03:26:17 2025
Only one PDF downloaded: C:\Users\clint\Desktop\RA Task\Downloads\180012bidtab.pdf
File size: 292.88 KB
Download time: Sun Jun  1 03:26:17 2025


In [36]:
# Download proposal document after BidTab
print("\n\nStarting Proposal document download...")

# Navigate to the Ohio DOT contracts website again
driver.get("https://contracts.dot.state.oh.us/document/documentSearchCriteria.do?from=topNav&newSearch=true")

# Wait for the page to load completely
wait = WebDriverWait(driver, 20)

try:
    print("Step 1: Looking for the dropdown containing Proposals option...")
    # Look for the dropdown containing Proposals option
    proposals_dropdown = wait.until(EC.presence_of_element_located((By.XPATH, "//select[option[@value='1003']]")))
    print("Found the dropdown, clicking...")
    
    proposals_dropdown.click()
    print("Clicked on the dropdown")
    
    # Select the Proposals option
    print("Step 2: Looking for the Proposals option...")
    proposals_option = wait.until(EC.presence_of_element_located((By.XPATH, "//option[@value='1003']")))
    print("Found the Proposals option, clicking...")
    
    proposals_option.click()
    print("Successfully selected Proposals option")
    
    # Find the PID input field and enter the same PID number
    print("Step 3: Looking for the PID input field...")
    pid_input = wait.until(EC.presence_of_element_located((By.NAME, "DP.PID_NUM.TEXT")))
    print("Found the PID input field, entering PID number...")
    
    pid_input.send_keys(pid_number)  # Use the same PID number as before
    print(f"Successfully entered PID number {pid_number}")
    
    # Take a screenshot before searching
    driver.save_screenshot(r"C:\Users\clint\Desktop\RA Task\Downloads\proposal_before_search.png")
    
    # Click the search button
    print("Step 4: Looking for the search button...")
    try:
        search_button = driver.find_element(By.XPATH, "//input[@type='submit' and @class='blue-button' and contains(@value, 'Search') and @onclick='return submitAction(this.form);']")
        print("Found search button using exact attributes")
    except Exception as e:
        print(f"Exact search button locator failed: {e}")
        search_button = driver.find_element(By.CLASS_NAME, "blue-button")
        print("Found search button by class name")
    
    # Scroll to the button and click it
    driver.execute_script("arguments[0].scrollIntoView(true);", search_button)
    search_button.click()
    print("Clicked the search button")
    
    # Wait for results to load
    print("Waiting for results page to load...")
    time.sleep(8)
    
    # Take a screenshot after search
    driver.save_screenshot(r"C:\Users\clint\Desktop\RA Task\Downloads\proposal_after_search.png")
    
    # Click on the "View" icon to download the PDF
    print("Step 5: Looking for the 'View' icon...")
    view_icon = wait.until(EC.presence_of_element_located((By.XPATH, "//img[@src='/images/open.gif']")))  
    print("Found the 'View' icon, clicking to download PDF...")
    
    view_icon.click()
    print("Clicked the view icon, Proposal PDF download started...")
    
    # Wait for the download to begin
    time.sleep(8)
    
except Exception as e:
    print(f"An error occurred while downloading the proposal: {e}")



Starting Proposal document download...
Step 1: Looking for the dropdown containing Proposals option...
Found the dropdown, clicking...
Clicked on the dropdown
Step 2: Looking for the Proposals option...
Found the Proposals option, clicking...
Successfully selected Proposals option
Step 3: Looking for the PID input field...
Found the PID input field, entering PID number...
Successfully entered PID number 76467
Step 4: Looking for the search button...
Found search button using exact attributes
Successfully selected Proposals option
Step 3: Looking for the PID input field...
Found the PID input field, entering PID number...
Successfully entered PID number 76467
Step 4: Looking for the search button...
Found search button using exact attributes
Clicked the search button
Waiting for results page to load...
Clicked the search button
Waiting for results page to load...
Step 5: Looking for the 'View' icon...
Found the 'View' icon, clicking to download PDF...
Clicked the view icon, Proposal P

In [37]:
# Close the browser when done
driver.quit()
print("Browser closed")

Browser closed


In [38]:
# Example: Download for a single PID (for testing)
# Uncomment the next line to test the download function with a single PID
# test_results = download_bid_documents("105522")  # Replace with a PID you want to test

In [40]:
# Batch download BidTabs and Proposals for all project IDs in the DataFrame
import glob
import pandas as pd
import time
from datetime import datetime

# Create a new DataFrame to store download results
results_df = pd.DataFrame(columns=[
    'project_id', 'download_time', 'bidtab_success', 'proposal_success',
    'bidtab_file', 'proposal_file', 'error'
])

# Get unique project IDs from the DataFrame
project_ids = df['project_id'].astype(str).unique()
print(f"Found {len(project_ids)} unique project IDs to process")

# Ask user if they want to proceed with all downloads
total_time_estimate = len(project_ids) * 2 * 60  # ~2 minutes per project ID
hours, remainder = divmod(total_time_estimate, 3600)
minutes, seconds = divmod(remainder, 60)
print(f"Estimated time to download all: {hours:.0f} hours, {minutes:.0f} minutes")

# Optional: limit to fewer items for testing
# Uncomment the next line to test with fewer items
# project_ids = project_ids[:3]  # Test with first 3 project IDs

# Loop through each project ID
for i, pid in enumerate(project_ids):
    start_time = datetime.now()
    print(f"\n--- Processing {i+1}/{len(project_ids)}: Project ID {pid} ---")
    
    try:
        # Download documents for this project ID
        download_results = download_bid_documents(str(pid), wait_time=5)
        
        # Record results
        results_df.loc[len(results_df)] = {
            'project_id': pid,
            'download_time': datetime.now(),
            'bidtab_success': download_results['bidtab'],
            'proposal_success': download_results['proposal'],
            'bidtab_file': download_results['bidtab_file'],
            'proposal_file': download_results['proposal_file'],
            'error': None
        }
        
        elapsed_time = (datetime.now() - start_time).total_seconds()
        print(f"Completed PID {pid} in {elapsed_time:.1f} seconds")
        
        # Save progress after each download
        results_df.to_csv(r"C:\Users\clint\Desktop\RA Task\Downloads\download_results.csv", index=False)
        
        # Add a short pause between downloads
        time.sleep(5)
        
    except Exception as e:
        print(f"Error processing PID {pid}: {e}")
        
        # Record error
        results_df.loc[len(results_df)] = {
            'project_id': pid,
            'download_time': datetime.now(),
            'bidtab_success': False,
            'proposal_success': False,
            'bidtab_file': None,
            'proposal_file': None,
            'error': str(e)
        }
        
        # Save progress even when errors occur
        results_df.to_csv(r"C:\Users\clint\Desktop\RA Task\Downloads\download_results.csv", index=False)

# Final results summary
print("\n--- Download Summary ---")
print(f"Total project IDs processed: {len(results_df)}")
print(f"BidTab downloads successful: {results_df['bidtab_success'].sum()}")
print(f"Proposal downloads successful: {results_df['proposal_success'].sum()}")
print(f"Errors encountered: {results_df['error'].notnull().sum()}")
results_path = r"C:\Users\clint\Desktop\RA Task\Downloads\download_results.csv"
print(f"Results saved to: {results_path}")

Found 202 unique project IDs to process
Estimated time to download all: 6 hours, 44 minutes

--- Processing 1/202: Project ID 105522 ---


BidTab download started for PID 105522
BidTab download started for PID 105522


Proposal download started for PID 105522
Proposal download started for PID 105522
Browser closed
Browser closed
Downloaded 2 new file(s):
Downloaded file: 180326.pdf
Downloaded file: 180326bidtab.pdf
Completed PID 105522 in 31.8 seconds
Downloaded 2 new file(s):
Downloaded file: 180326.pdf
Downloaded file: 180326bidtab.pdf
Completed PID 105522 in 31.8 seconds

--- Processing 2/202: Project ID 88832 ---

--- Processing 2/202: Project ID 88832 ---


BidTab download started for PID 88832
BidTab download started for PID 88832


Proposal download started for PID 88832
Proposal download started for PID 88832
Browser closed
Browser closed
Downloaded 2 new file(s):
Downloaded file: 180569bidtab.pdf
Downloaded file: 180569.pdf
Completed PID 88832 in 36.2 seconds
Downloaded 2 new 

# Ohio DOT PDF Downloader

This notebook demonstrates how to:

1. Navigate to the Ohio Department of Transportation contracts website
2. Search for documents using Project ID (PID) numbers
3. Download both BidTab and Proposal PDFs automatically
4. Process multiple Project IDs from a DataFrame

## Key Features

- Handles multiple PID numbers from a DataFrame
- Downloads both BidTab and Proposal documents for each PID
- Robust error handling with multiple fallback methods for finding web elements
- Progress tracking and result logging to CSV
- Verifies successful downloads

The notebook first defines a function that can download documents for a single PID, then uses that function to process all PIDs in the DataFrame.